In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from features import featureExtractionPipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from tqdm import tqdm_notebook as tqdm
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.stats import moment
import statsmodels.api as sm
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import pyeeg
import glob
import pywt

In [6]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]



    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [7]:
feature_array = []
files = glob.glob('../data/2colours_muse_transition/*.csv')
files += glob.glob('../data/2colours_muse_onlycolor/*.csv')

# PER FILE

In [ ]:
feature_array = []
label_array = []
for file in tqdm(files):
    per_file_features = []
    data = pd.read_csv(file,header=None)
    channels = data[:-1]
    label = data.iloc[-1]
    label = int(list(set(label))[0])
    for i in range(len(channels)):
        channel = channels.iloc[i].values
        channel_features = featureExtractionPipeline(channel,fs=256).runPipeline()
        per_file_features.append(channel_features)
    label_array.append(label)
    feature_array.append(per_file_features)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(feature_array,label_array)

y_train = OneHotEncoder(sparse=True).fit_transform(np.array(y_train).reshape(len(y_train),1))
y_test = OneHotEncoder(sparse=True).fit_transform(np.array(y_test).reshape(len(y_test),1))

X_train_flat = np.array(X_train).flatten()
X_train_flat = X_train_flat.reshape(len(X_train_flat),1)
X_test_flat = np.array(X_test).flatten()
X_test_flat = X_test_flat.reshape(len(X_test_flat),1)


scaler = StandardScaler()

X_train_flat_scaled = scaler.fit_transform(X_train_flat)
X_train_scaled = X_train_flat_scaled.flatten().reshape(np.array(X_train).shape)
X_train_scaled = X_train_scaled.reshape(len(X_train_scaled),4,29,1)

X_test_flat_scaled = scaler.fit_transform(X_test_flat)
X_test_scaled = X_test_flat.flatten().reshape(np.array(X_test).shape)
X_test_scaled = X_test_scaled.reshape(len(X_test_scaled),4,29,1)

In [ ]:
'''
for i in range(len(y_train)):
    if y_train[i] == 2:
        y_train[i] = 0
        
for i in range(len(y_test)):
    if y_test[i] == 2:
        y_test[i] = 0
'''

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()
model.add(Conv2D(32, kernel_size=2, activation='relu', input_shape=(4,29,1)))
model.add(Conv2D(16, kernel_size=2, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train_scaled,y_train,validation_split=0.2)